In [1]:
!pip install ultralytics -q
!pip install roboflow -q

DEPRECATION: Loading egg at c:\users\kiit\appdata\local\programs\python\python311\lib\site-packages\mathparse-0.1.2-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\kiit\appdata\local\programs\python\python311\lib\site-packages\sqlalchemy-1.3.24-py3.11-win-amd64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
DEPRECATION: Loading egg at c:\users\kiit\appdata\local\programs\python\python311\lib\site-packages\mathparse-0.1.2-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for

In [2]:
from ultralytics.nn.modules import (
    AIFI,
    C1,
    C2,
    C2PSA,
    C3,
    C3TR,
    ELAN1,
    OBB,
    PSA,
    SPP,
    SPPELAN,
    SPPF,
    A2C2f,
    AConv,
    ADown,
    Bottleneck,
    BottleneckCSP,
    C2f,
    C2fAttn,
    C2fCIB,
    C2fPSA,
    C3Ghost,
    C3k2,
    C3x,
    CBFuse,
    CBLinear,
    Classify,
    Concat,
    Conv,
    Conv2,
    ConvTranspose,
    Detect,
    DWConv,
    DWConvTranspose2d,
    Focus,
    GhostBottleneck,
    GhostConv,
    HGBlock,
    HGStem,
    ImagePoolingAttn,
    Index,
    LRPCHead,
    Pose,
    RepC3,
    RepConv,
    RepNCSPELAN4,
    RepVGGDW,
    ResNetLayer,
    RTDETRDecoder,
    SCDown,
    Segment,
    TorchVision,
    WorldDetect,
    YOLOEDetect,
    YOLOESegment,
    v10Detect,
)

WARNING Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\KIIT\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [34]:
import configparser
config = configparser.ConfigParser()
config.read('../../myconfig.config')
print(config['roboflow']['roboflow_api_key'])


u8Ur5ywKLswqwt0uMTXl


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key=config['roboflow']['roboflow_api_key'])
project = rf.workspace("poacher-ifyuf").project("poacher")
version = project.version(2)
dataset = version.download("yolov8",location="..\..\Architecture\Test_Dataset")

In [ ]:
# import sys
# sys.path.append('/content/drive/MyDrive/Colab_Notebooks/YOLO_architecture/utillity')

In [36]:
from ultralytics import nn
from ultralytics.nn import modules, tasks
import ultralytics.nn.autobackend as autobackend
from model_classes import Upsample, Conv, C2f, SPPF, DFL, Head, Bottleneck, Concat
from ultralytics.nn.tasks import parse_model, yaml_model_load
from copy import deepcopy


def patch_ultralytics_layers(custom_classes):
    submodules = [
        nn,                          # ultralytics.nn
        modules,                     # ultralytics.nn.modules
        tasks,                       # ultralytics.nn.tasks
        autobackend,                 # ultralytics.nn.autobackend
    ]

    for cls in custom_classes:
        for mod in submodules:
            mod.__dict__[cls.__name__] = cls

def build_model(model_yaml, version='n', ch=3, verbose=False, custom_classes=[]):

  patch_ultralytics_layers(custom_classes)

  mod_ya = model_yaml if isinstance(model_yaml, dict) else yaml_model_load(model_yaml)
  mod_ya['scale'] = version.lower()
  model, save  = parse_model(deepcopy(mod_ya), ch=ch, verbose=verbose)

  return model,save


In [38]:
from ultralytics import YOLO
m_class = [Upsample, Conv, C2f, SPPF, DFL, Head, Bottleneck, Concat]
model_yaml = "my_yolov8l.yaml"

model,save = build_model(model_yaml, custom_classes=m_class, version='l', verbose=False)

model = YOLO(model_yaml, verbose=True)
print(f"{sum(p.numel() for p in model.parameters()) / 1e6:.2f} million parameters")


                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  model_classes.Conv                           [3, 64, 3, 2]                 
  1                  -1  1     73984  model_classes.Conv                           [64, 128, 3, 2]               
  2                  -1  3    279808  model_classes.C2f                            [128, 128, 3, True]           
  3                  -1  1    295424  model_classes.Conv                           [128, 256, 3, 2]              
  4                  -1  6   2101248  model_classes.C2f                            [256, 256, 6, True]           
  5                  -1  1   1180672  model_classes.Conv                           [256, 512, 3, 2]              
  6                  -1  6   8396800  model_classes.C2f                            [512, 512, 6, True]           
  7                  -1  1   2360320  model_classes.Conv                           [512

In [39]:
for m in model.modules():
  if isinstance(m, (Conv, Conv2, DWConv)) and hasattr(m, "bn"):
    if isinstance(m, Conv2):
      pass
    print(m.conv)
    break


Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)


In [ ]:
results = model.train(data="/content/poacher-2/data.yaml", epochs=1, imgsz=64, device=0 )